**Langkah 1: Memuat dan Memahami Dataset**

Sebelum memulai eksperimen, kita memuat dataset wine quality dan melakukan pra-pemrosesan dasar, termasuk normalisasi fitur dan pembagian data menjadi set pelatihan dan pengujian.

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Memuat dataset
file_path = '/content/winequality-red.csv'  # Ganti dengan path dataset Anda
data = pd.read_csv(file_path, sep=';')

# Menampilkan beberapa baris dataset untuk memahami strukturnya
print(data.head())

# Menampilkan informasi dataset
print(data.info())

# Memisahkan fitur (X) dan target (y)
X = data.drop(columns=['quality'])
y = data['quality']

# Normalisasi fitur menggunakan StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Menyesuaikan nilai target agar dimulai dari 0
y_adjusted = y - y.min()

# Perbarui pembagian dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_adjusted, test_size=0.2, random_state=42, stratify=y_adjusted)


   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

**Penjelasan Output:**

Data berhasil dimuat, terdiri dari 11 fitur input (seperti acidity, alcohol, dan lainnya) dan 1 target output (quality).

**Langkah 2: Membangun Model RNN**

Model RNN dirancang untuk menangkap hubungan sekuensial dalam data. Pada kasus ini, kita menggunakan RNN sederhana dengan opsi pooling (MaxPooling atau AvgPooling) untuk menghasilkan representasi yang lebih ringkas.

In [2]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, pooling_type="max"):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.pooling_type = pooling_type
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x.unsqueeze(1))  # Tambahkan dimensi waktu
        if self.pooling_type == "max":
            out, _ = torch.max(out, dim=1)
        else:  # AvgPooling
            out = torch.mean(out, dim=1)
        out = self.fc(out)
        return out


**Penjelasan:**

- `input_size`: Jumlah fitur dalam input (11 untuk dataset wine).
- `hidden_size`: Ukuran lapisan tersembunyi yang akan diuji pada eksperimen.
- `pooling_type`: Tipe pooling untuk merangkum informasi dari output RNN.

**Langkah 3: Training dan Evaluasi**

Pada langkah ini, kita melatih model menggunakan kombinasi parameter seperti hidden size, pooling type, optimizer, dan epoch. Kemudian juga menggunakan scheduler untuk mengatur learning rate.

In [6]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Konversi data ke tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# DataLoader untuk batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Fungsi evaluasi
def evaluate_model(model, dataloader):
    model.eval()
    y_pred, y_true = [], []
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.tolist())
            y_true.extend(y_batch.tolist())
    return accuracy_score(y_true, y_pred)

# Parameter eksperimen
input_size = X_train.shape[1]
hidden_sizes = [16, 32, 64]
pooling_types = ["max", "avg"]
optimizers = ["SGD", "RMSProp", "Adam"]
num_epochs_list = [5, 50, 100, 250, 350]

# Loop untuk eksperimen
for hidden_size in hidden_sizes:
    for pooling_type in pooling_types:
        for opt_name in optimizers:
            # Model, optimizer, dan scheduler
            model = RNNModel(input_size=input_size, hidden_size=hidden_size, num_classes=len(y.unique()), pooling_type=pooling_type)
            if opt_name == "SGD":
                optimizer = optim.SGD(model.parameters(), lr=0.01)
            elif opt_name == "RMSProp":
                optimizer = optim.RMSprop(model.parameters(), lr=0.01)
            elif opt_name == "Adam":
                optimizer = optim.Adam(model.parameters(), lr=0.001)
            criterion = nn.CrossEntropyLoss()
            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

            # Training loop
            for num_epochs in num_epochs_list:
                model.train()
                for epoch in range(num_epochs):
                    for X_batch, y_batch in train_loader:
                        optimizer.zero_grad()
                        outputs = model(X_batch)
                        loss = criterion(outputs, y_batch)
                        loss.backward()
                        optimizer.step()
                    scheduler.step()

                # Evaluasi
                accuracy = evaluate_model(model, test_loader)
                print(f"Hidden: {hidden_size}, Pool: {pooling_type}, Opt: {opt_name}, Epochs: {num_epochs}, Acc: {accuracy:.4f}")


Hidden: 16, Pool: max, Opt: SGD, Epochs: 5, Acc: 0.5188
Hidden: 16, Pool: max, Opt: SGD, Epochs: 50, Acc: 0.5844
Hidden: 16, Pool: max, Opt: SGD, Epochs: 100, Acc: 0.5844
Hidden: 16, Pool: max, Opt: SGD, Epochs: 250, Acc: 0.5844
Hidden: 16, Pool: max, Opt: SGD, Epochs: 350, Acc: 0.5844
Hidden: 16, Pool: max, Opt: RMSProp, Epochs: 5, Acc: 0.6094
Hidden: 16, Pool: max, Opt: RMSProp, Epochs: 50, Acc: 0.5906
Hidden: 16, Pool: max, Opt: RMSProp, Epochs: 100, Acc: 0.6000
Hidden: 16, Pool: max, Opt: RMSProp, Epochs: 250, Acc: 0.6000
Hidden: 16, Pool: max, Opt: RMSProp, Epochs: 350, Acc: 0.6000
Hidden: 16, Pool: max, Opt: Adam, Epochs: 5, Acc: 0.5500
Hidden: 16, Pool: max, Opt: Adam, Epochs: 50, Acc: 0.6031
Hidden: 16, Pool: max, Opt: Adam, Epochs: 100, Acc: 0.6062
Hidden: 16, Pool: max, Opt: Adam, Epochs: 250, Acc: 0.6062
Hidden: 16, Pool: max, Opt: Adam, Epochs: 350, Acc: 0.6062
Hidden: 16, Pool: avg, Opt: SGD, Epochs: 5, Acc: 0.5719
Hidden: 16, Pool: avg, Opt: SGD, Epochs: 50, Acc: 0.6031
H

1. Tren Akurasi Berdasarkan Hidden Size
- Hidden Size 16:
  - Pooling Avg dengan optimizer Adam mencapai akurasi 60.62% pada 350 epoch.
  - Pooling Max dengan optimizer RMSProp mencapai hasil terbaik 60.00%
  - SGD memberikan hasil yang stagnan di 58.44% setelah 50 epoch.

- Hidden Size 32:
  - Pooling Avg dengan optimizer RMSProp menunjukkan performa terbaik dengan akurasi 63.75% pada 50 epoch.
  - Pooling Max dengan optimizer RMSProp juga memberikan hasil konsisten hingga 61.25%.

- Hidden Size 64:
  - Pooling Avg dengan optimizer RMSProp memberikan hasil terbaik dari seluruh eksperimen, mencapai akurasi 65.62% pada 50 epoch.
  - Pooling Max dengan optimizer Adam mencapai 60.94%, sedikit lebih rendah dari AvgPooling.

2. Pooling Type
- AvgPooling:
  - Konsisten memberikan akurasi lebih tinggi dibandingkan MaxPooling, terutama pada kombinasi hidden size besar dan optimizer RMSProp.
  - Performa terbaik dengan hidden size 64 dan RMSProp.

- MaxPooling:
  - Sedikit lebih lemah dibandingkan AvgPooling, tetapi memberikan hasil yang stabil pada kombinasi tertentu (misalnya, hidden size 16 dan Adam).
  
3. Optimizer
- RMSProp:
  - Memberikan hasil terbaik secara keseluruhan.
  - Kombinasi Hidden Size 64 + AvgPooling + RMSProp mencapai akurasi tertinggi 65.62%.

- Adam:
  - Hasil cukup baik, tetapi sering lebih rendah dari RMSProp.
  - Performa terbaik pada hidden size 32 dan 64 dengan akurasi sekitar 60%-61%.

- SGD:
  - Hasil yang paling lemah, sering stagnan di sekitar 58%-59%.

4. Epoch
- Jumlah Epoch Optimal:
  - Akurasi terbaik sering dicapai dalam 50-100 epoch.
  - Penambahan epoch hingga 350 tidak menunjukkan peningkatan signifikan.